In [1]:
from lxml import etree
from lxml.builder import E

mx_graph_tree = etree.parse('./Example Files/mxgraph.xml')

In [2]:
# Get all features for the feature model
def get_features(mx_graph):
    features = {}

    for feature in mx_graph.xpath('''//root | 
                                     //concrete | 
                                     //abstract'''):
        feature_dict = dict(feature.items())
        
        if feature_dict and 'clon' not in feature_dict['id']:
            features[feature_dict['id']] = {
                'type': feature_dict['type'],
                'label': feature_dict['label'],
            }
            
    return features

features = get_features(mx_graph_tree)
len(features)

25

In [3]:
# Get all relations for the feature model
def get_relations(mx_graph):
    relations = []

    for relation in mx_graph_tree.xpath('''//rel_concrete_root | 
                                           //rel_abstract_root | 
                                           //rel_concrete_abstract | 
                                           //rel_abstract_abstract |
                                           //rel_concrete_concrete'''):

        relation_dict = dict(relation.items())
        relation_source_dict = dict(relation.find('.//mxCell').items())

        relations.append({
            'type': relation_dict['relType'],
            'source': relation_source_dict['source'],
            'target': relation_source_dict['target'],
        })
        
    return relations

relations = get_relations(mx_graph_tree)
len(relations)

27

In [41]:
def get_root(features):
    for key, feature in features.items():
        if feature['type'] == 'root':
            return key

root_id = get_root(features)
root_id

'1'

In [42]:
build_rels_funcs = {
    'root': lambda x: f'\tF{x} #= 1,',
    'optional': lambda x, y: f'\tF{x} #>= F{y},',
    'mandatory': lambda x, y: f'\tF{x} #= F{y},',
    'requires': lambda x, y: f'\tF{x} #==> F{y},',
    'excludes': lambda x, y: f'\tF{x} * F{y} #= 0,',
}

function_signature = "function(L):-"
variables = ''.join(['\tL=[', ', '.join([f'F{k}' for k in features.keys()]) , '],'])
variables_domain = '\tfd_domain(L, 0, 1),'

root_constrain = [build_rels_funcs['root'](root_id)]
constrains = '\n'.join(root_constrain + [
    build_rels_funcs[relation['type']](relation['source'], relation['target']) for relation in relations
])

labeling = '\tfd_labneling(L)'

prolog_str = '\n'.join(
    [
        function_signature,
        variables,
        variables_domain,
        constrains,
        labeling,
    ]
)

prolog_str = prolog_str + '.'

In [43]:
print(prolog_str)

function(L):-
	L=[F1, F2, F3, F6, F49, F60, F72, F75, F77, F113, F114, F115, F141, F142, F143, F144, F145, F205, F206, F207, F208, F251, F256, F263, F270],
	fd_domain(L, 0, 1),
	F1 #= 1,
	F2 #>= F1,
	F3 #>= F1,
	F6 #= F1,
	F49 #= F1,
	F60 #>= F1,
	F75 #==> F77,
	F208 #==> F72,
	F206 #==> F143,
	F251 #>= F1,
	F113 #= F251,
	F115 #>= F251,
	F114 #>= F251,
	F141 #>= F256,
	F144 #= F256,
	F145 #>= F256,
	F142 #= F256,
	F143 #>= F256,
	F256 #= F1,
	F205 #= F263,
	F206 #>= F263,
	F207 #>= F263,
	F208 #>= F263,
	F263 #>= F1,
	F77 #>= F270,
	F72 #= F270,
	F75 #>= F270,
	F270 #>= F1,
	fd_labneling(L).
